## Comparing the plotting and models interface

In this exercise we want to ... 
Compare plotting and models with one example   
https://rebeccabilbro.github.io/interactive-viz-bokeh/

#### Loading our dataset

In [1]:
# importing the necessary dependencies
import numpy as np
import pandas as pd

In [2]:
# make bokeh display figures inside the notebook
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

**Note:**   
The cell above allows us to plot the bokeh visualizations inline in the notebook. By default it will open a new tab in your browser window with the plot.

In [3]:
# loading the Dataset with geoplotlib
dataset = pd.read_csv('./data/world_population.csv', index_col=0)

In [4]:
# looking at the dataset
dataset.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,307.972222,312.366667,314.983333,316.827778,318.666667,320.622222,...,562.322222,563.011111,563.422222,564.427778,566.311111,568.850000,571.783333,574.672222,577.161111,NaN
Andorra,AND,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,30.587234,32.714894,34.914894,37.170213,39.470213,41.800000,...,180.591489,182.161702,181.859574,179.614894,175.161702,168.757447,161.493617,154.863830,149.942553,NaN
Afghanistan,AFG,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,14.038148,14.312061,14.599692,14.901579,15.218206,15.545203,...,39.637202,40.634655,41.674005,42.830327,44.127634,45.533197,46.997059,48.444546,49.821649,NaN
Angola,AGO,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,4.305195,4.384299,4.464433,4.544558,4.624228,4.703271,...,15.387749,15.915819,16.459536,17.020898,17.600302,18.196544,18.808215,19.433323,20.070565,NaN
Albania,ALB,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,60.576642,62.456898,64.329234,66.209307,68.058066,69.874927,...,108.394781,107.566204,106.843759,106.314635,106.013869,105.848431,105.717226,105.607810,105.444051,NaN


---

#### Using the plotting interface

In [5]:
# importing the plotting dependencies 
from bokeh.plotting import figure, show

Using the knowledge from the first exercise, we'll create another visualization that compares two values against each other.   
In this example, we'll take the mean population density for each year and compare its growth to the one of Japan.   
In this first task, we'll create our plot with the high-level plotting interface of Bokeh. This will allow us to leverage the abstraction of this interface to quickly and easily build a visualization without too much configuration, as seen in the first exercise of this lesson.   
Once we have our plot in place, we'll recreate the same one with the models interface to display the differences between the two interaces.

In [6]:
# preparing our data of the mean values per year and Japan
years = [year for year in dataset.columns if not year[0].isalpha()]
mean_pop_vals = [np.mean(dataset[year]) for year in years]
jp_vals = [dataset.loc[['Japan']][year] for year in years]

In [54]:
# plotting the global population density change and the one for Japan 
plot = figure(title='Gloabal Mean Population Density compared to Japan', x_axis_label='Year', y_axis_label='Population Density')

plot.line(years, mean_pop_vals, line_width=2, legend='Global Mean')
plot.cross(years, jp_vals, legend='Japan', line_color='red')

show(plot)

**Note:**   
Note that the amount of data points in the first and second argument passed to the plotting methods has to be the same.   
If your `x` list has 10 values, your `y` list also has to have 10 values.

---

#### Using the models interface

In [127]:
# importing the models dependencies 
from bokeh.io import show
from bokeh.models.grids import Grid
from bokeh.models.plots import Plot 
from bokeh.models.axes import LinearAxis
from bokeh.models.ranges import Range1d
from bokeh.models.glyphs import Line, Cross
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tickers import SingleIntervalTicker, YearsTicker
from bokeh.models.renderers import GlyphRenderer
from bokeh.models.annotations import Title, Legend, LegendItem

**Note:**   
Even though we are defining all the sub-packages for the imports here, we could also simply use the top level package `models` for each import.   
However, keep in mind that this might lead to problems of wrong import references since there might be more than one definition of an element in the bokeh package.   
For example the `show` mehtod is defined in the `bokeh.io` and `bokeh.plotting` packages.   

All the sub-packages of the models interface can be found here:   
http://bokeh.pydata.org/en/latest/docs/reference/models.html

Before we build our plot, we have to find out the `min` and `max` values for the y axis since we don't want to have a too large or small range of values.   
For the x axis, we have our list of years pre-defined.

In [138]:
# defining the range for the x and y axis
extracted_mean_pop_vals = [val for i, val in enumerate(mean_pop_vals)
                    if i not in [0, len(mean_pop_vals) - 1]]
extracted_jp_vals = [jp_val['Japan'] for i, jp_val in enumerate(jp_vals)
                    if i not in [0, len(jp_vals) - 1]]

min_pop_density = min(extracted_mean_pop_vals)
min_jp_densitiy = min(extracted_jp_vals)
min_y = int(min(min_pop_density, min_jp_densitiy))

max_pop_density = max(extracted_mean_pop_vals)
max_jp_densitiy = max(extracted_jp_vals)
max_y = int(max(max_jp_densitiy, max_pop_density))

xdr = Range1d(int(years[0]), int(years[-1]))
ydr = Range1d(min_y, max_y)

Once we have the min and max values for the y axis, we can create two `Axis` objects that will be used to display the axis lines and the label for the axis.   
Since we also want ticks between the different values, we have to pass in a `Ticker` object that creates this setup for us.

In [139]:
# creating the axis
axis_def = dict(
    axis_line_color='#222222',
    axis_line_width=1,
    major_tick_line_color='#222222',
    major_label_text_color='#222222',
    major_tick_line_width=1,
)

x_axis = LinearAxis(
    ticker = SingleIntervalTicker(interval=10),
    axis_label = 'Year',
    **axis_def
)

y_axis = LinearAxis(
    ticker = SingleIntervalTicker(interval=50),
    axis_label = 'Population Density',
    **axis_def
)

Creating the title and plot itself are straight forward. We can pass a `Title` object to the title attribute of the `Plot` object.

In [140]:
# creating the plot object
title = Title(
    align = 'left',
    text = 'Gloabal Mean Population Density compared to Japan'
)

plot = Plot(
    x_range=xdr,
    y_range=ydr,
    plot_width=650,
    plot_height=600,
    title=title
)

When working with data, we always need to insert our data into a `DataSource` object. This can then be used to map the data source to the `Glyph` object that will be displayed in the plot.

In [141]:
# creating the data display
line_source = ColumnDataSource(dict(
    x=years,
    y=mean_pop_vals
))

line_glyph = Line(x='x', y='y', line_color='#2678b2', line_width=2)

cross_source = ColumnDataSource(dict(
    x=years,
    y=jp_vals
))

cross_glyph = Cross(x='x', y='y', line_color='#fc1d26')

When adding objects to the plot, have to use the right add method.   
For layout elements like the `Axis` objects, we have to use the `add_layout`method.   

`Glyphs`, that display our data have to be added with the `add_glyph` method.

In [142]:
# assembling the plot
plot.add_layout(x_axis, 'below')
plot.add_layout(y_axis, 'left')

plot.add_glyph(line_source, line_glyph)
plot.add_glyph(cross_source, cross_glyph)

line_renderer = plot.renderers[2]
cross_renderer = plot.renderers[3]

In [143]:
show(plot)

In order to add a legend to our plot, we again have to use an object.   
Each `LegendItem` object will be displayed in one line in the legend.

In [144]:
# creating the legend
legend_items= [
    LegendItem(label='Gloabal Mean', renderers=[line_renderer]),
    LegendItem(label='Japan', renderers=[cross_renderer])
]

legend = Legend(
    items=legend_items,
    location='top_right'
)

Creating the grid is straightforward, we simply have to instantiate two `Grid`objects for the x and y axis. These grids will get the tickers of the previously created x and y axis.

In [145]:
# creating the grid
x_grid = Grid(dimension=0, ticker=x_axis.ticker)
y_grid = Grid(dimension=1, ticker=y_axis.ticker)

To add the last final touches, we, again, use the `add_layout` method to add the grid and the legend to our plot.   

After this, we can finally display our complete plot that will look like the one we've created in the first task with only 4 lines of code.

In [146]:
# adding the legend and grids to the plot
plot.add_layout(legend)
plot.add_layout(x_grid)
plot.add_layout(y_grid)

show(plot)

As we can really clearly see here, we get nearly the same result.   
The models interface is not convenient for simple plots like the one we created here.   
Later in this lesson, we will use some elements for interactions from the models interface, however, setting up a whole plot with the **models interface is not recommended**.